In [1]:
from threeML import *

import numpy as np
import pandas as pd

Welcome to JupyROOT 6.22/02


23:09:01 WARNING   The naima package is not available. Models that depend on it will not be         ]8;id=854908;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=346029;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/astromodels/functions/functions_1D/functions.py#49\49]8;;\
                  available                                                                                        

         WARNING   The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it  ]8;id=404208;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=312221;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/astromodels/functions/functions_1D/functions.py#70\70]8;;\
                  will not be available.                                                                           

23:09:02 WARNING   The ebltable package is not available. Models that depend on it will not be     ]8;id=936095;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/astromodels/functions/functions_1D/absorption.py\absorption.py]8;;\:]8;id=605237;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/astromodels/functions/functions_1D/absorption.py#37\37]8;;\
                  available                                                                                        

         WARNING   We have set the min_value of K to 1e-99 because there was a postive transform   ]8;id=779144;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/astromodels/core/parameter.py\parameter.py]8;;\:]8;id=615766;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/astromodels/core/parameter.py#706\706]8;;\

...

23:09:13 WARNING   The cthreeML package is not installed. You will not be able to use plugins which  ]8;id=982673;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=665965;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/__init__.py#80\80]8;;\
                  require the C/C++ interface (currently HAWC)                                                     

         WARNING   Could not import plugin HAWCLike.py. Do you have the relative instrument         ]8;id=537416;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=659911;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/__init__.py#130\130]8;;\
                  software installed and configured?                                                               

23:09:19 WARNING   Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=482219;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=460989;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/__init__.py#343\343]8;;\
                  performances in 3ML                                                                              

23:09:20 WARNING   Env. variable MKL_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=328579;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=326269;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/__init__.py#343\343]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable NUMEXPR_NUM_THREADS is not set. Please set it to 1 for optimal     ]8;id=249507;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=732906;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/__init__.py#343\343]8;;\
                  performances in 3ML                                                                              

In [70]:
lle_catalog = FermiLLEBurstCatalog()

def generate_lle_lightcurve(name = "GRB141222298", time_in = -30, time_out = 30, dt=.1, returnfig = False):
    """
    Parameters: name, time_in, time_out, dt, 
    returnfig = True to return output as tuple with figure

    Returns a dataframe with binning and counts;
    figure optionally.
    """
    lle_catalog.query_sources(name)

    dload = download_LLE_trigger_data(name)

    ts_lle = TimeSeriesBuilder.from_lat_lle(
        "GRB141222298",
        lle_file=dload["lle"],
        ft2_file=dload["ft2"],
        rsp_file=dload['rsp']
    )

    ts_lle.create_time_bins(time_in, time_out, dt=dt)

    data = ts_lle.bins._create_pandas()
    data['Counts'] = ts_lle.total_counts_per_interval

    if returnfig is True:
        return data, ts_lle.view_lightcurve(time_in, time_out, dt=dt)
    else:
        return data

In [71]:
gbm_catalog = FermiGBMBurstCatalog()

def generate_gbm_lightcurve(name = "GRB141222298", time_in = -30, time_out = 30, dt=.1, detectors = None, returnfig = False):
    """
    Parameters: name, time_in, time_out, dt, 
    detectors = None for default mask; list for specific detectors
    returnfig = True to return output as tuple with figure

    Returns a dictionary with dataframes with binning and counts for each detector;
    figures optionally.
    """
    gbm_catalog.query_sources(name)

    grb_info = gbm_catalog.get_detector_information()['GRB%s'%name]

    if gbm_detectors is None:
        gbm_detectors = grb_info["detectors"]
    print(gbm_detectors)

    dload = download_GBM_trigger_data('bn%s'%name[3:], detectors=gbm_detectors)

    if returnfig is True:
        figures = []

    time_series = {}
    for det in gbm_detectors:

        # Calculate background
        ts_cspec = TimeSeriesBuilder.from_gbm_cspec_or_ctime(
            det, cspec_or_ctime_file=dload[det]["cspec"], rsp_file=dload[det]["rsp"]
        )
        ts_cspec.set_background_interval(*background_interval.split(","))
        ts_cspec.save_background(f"{det}_bkg.h5", overwrite=True)

        # Build gbm timeseries
        ts_tte = TimeSeriesBuilder.from_gbm_tte(
            det,
            tte_file=dload[det]["tte"],
            rsp_file=dload[det]["rsp"],
            restore_background=f"{det}_bkg.h5",
        )

        # Bin data
        ts_tte.create_time_bins(time_in, time_out, dt=dt)

        # Generate dataframe
        data = ts_tte.bins._create_pandas()
        data['Counts'] = ts_tte.total_counts_per_interval

        time_series[det] = data

        if returnfig is True:
            figures.append(ts_tte.view_lightcurve(time_in, time_out, dt = dt))

    if returnfig is True:
        return time_series, figures
    else:
        return time_series

23:54:10 INFO      Building cache for fermigbrst                                 ]8;id=643984;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/io/get_heasarc_table_as_pandas.py\get_heasarc_table_as_pandas.py]8;;\:]8;id=679631;file:///Users/xboluna/opt/miniconda3/envs/threeML/lib/python3.7/site-packages/threeML/io/get_heasarc_table_as_pandas.py#112\112]8;;\